In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load training data
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
# Rename the *question_text* column for convenience
train_df = train_df.rename({'question_text': 'question'}, axis=1)

In [ ]:
train_df['question'].isnull().sum()

In [ ]:
train_df['question'].isna().sum()

In [ ]:
train_df.columns

In [ ]:
train_df.target.value_counts()

In [ ]:
import matplotlib.ticker as ticker

ncount = train_df.shape[0]

plt.figure(figsize=(7, 5))

ax = sns.countplot(data=train_df, x='target')
plt.title('Portion of Questions')
plt.xlabel('Number of Axles')

# Make twin axis
ax2=ax.twinx()

# Switch so count axis is on right, frequency on left
ax2.yaxis.tick_left()
ax.yaxis.tick_right()

# Also switch the labels over
ax.yaxis.set_label_position('right')
ax2.yaxis.set_label_position('left')

ax2.set_ylabel('Frequency [%]')

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
            ha='center', va='bottom') # set the alignment of the text

# Use a LinearLocator to ensure the correct number of ticks
ax.yaxis.set_major_locator(ticker.LinearLocator(11))

# Fix the frequency range to 0-100
ax2.set_ylim(0,100)
ax.set_ylim(0,ncount)

# And use a MultipleLocator to ensure a tick spacing of 10
ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# Need to turn the grid on ax2 off, otherwise the gridlines end up on top of the bars
ax2.grid(None)

In [ ]:
pd.set_option('display.max_columns', None)
insincere_qes = train_df[train_df['target'] == 1]
print(insincere_qes[-5:].question.to_string())

In [ ]:
pd.set_option('display.max_columns', None)
sincere_qes = train_df[train_df['target'] == 0]
print(sincere_qes[-5:].question)

## **Words cloud**

In [ ]:
from wordcloud import WordCloud

In [ ]:
print('Word cloud image generated from insincere questions')
insincere_wordcloud = WordCloud(width=800, height=400, background_color ='black', min_font_size = 10).generate(str(train_df[train_df["target"] == 1]["question"]))
#Positive Word cloud
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(insincere_wordcloud)
plt.axis("off")
# plt.tight_layout(pad=0)
plt.show();

In [ ]:
print('Word cloud image generated from sincere questions')
sincere_wordcloud = WordCloud(width=600, height=400, background_color ='black', min_font_size = 10).generate(str(train_df[train_df["target"] == 0]["question"]))
#Positive Word cloud
plt.figure(figsize=(15,6), facecolor=None)
plt.imshow(sincere_wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show();

## **Text statistics** 

In [ ]:
train_df.head()

In [ ]:
import string
from nltk.corpus import stopwords

# Set up contraction dictionary
contraction_dict = {"dont": "do not", "aint": "is not", "isnt": "is not", "doesnt": "does not", "cant": "cannot", "mustnt": "must not", "hasnt": "has not", "havent": "have not", "arent": "are not", "ain't": "is not", "aren't": "are not","can't": "cannot", "‘cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "Iam": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

# Set up stop words list
stop_words = stopwords.words('english')
stop_words.remove('not')

# Set up puntuation list
punctuation = string.punctuation

****Feature engineering****

In [ ]:
#Feature Engineering on train_df data

def create_features(df):
    """Retrieve from the text column the number of: characters, words, unique words, stopwords,
    punctuations, upper/lower case chars"""
    df["lenght"] = df["question"].apply(lambda x: len(str(x)))
    df["no_words"] = df["question"].apply(lambda x: len(x.split()))
    df["no_unique_words"] = df["question"].apply(lambda x: len(set(str(x).split())))
    df["no_stopwords"] = df["question"].apply(lambda x : len([nw for nw in str(x).split() if nw.lower() in stop_words]))
    df["no_punctuation"] = df["question"].apply(lambda x : len([np for np in str(x) if np in punctuation]))
    df["no_uppercase"] = df["question"].apply(lambda x : len([nu for nu in str(x).split() if nu.isupper()]))
    df["no_lowercase"] = df["question"].apply(lambda x : len([nl for nl in str(x).split() if nl.islower()]))
    return df

In [ ]:
train_df = create_features(train_df)

The statistics of insincere questions

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train_df[train_df['target'] == 0].describe()

The statistics of sincere questions

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train_df[train_df['target'] == 1].describe()

In [ ]:
# num_feat = ['lenght', 'no_unique_words', 'no_stopwords', 
#             'no_punctuation', 'no_uppercase', 'no_lowercase', 'target'] 
# # side note : remove target if needed later

# dfsample = train_df[num_feat].sample(n=round(train_df.shape[0]/6), random_state=42)

# plt.figure(figsize=(15,15))
# sns.set_context("paper", rc={"axes.labelsize":16})
# sns.pairplot(data=dfsample, hue='target')
# plt.show()

****The distribution of the questions's number of lowercases****

In [ ]:
plt.figure(figsize=(10, 10))
sns.displot(train_df, x='no_lowercase', hue='target', kind='hist', bins=50)
plt.title("Distribution of the question's number of lowercases")

In [ ]:
target_correlation = train_df.corr()['target'][1:]

In [ ]:
# mask = np.zeros_like(train_df[num_feat].corr(), dtype=np.bool) 
# mask[np.triu_indices_from(mask)] = True 

# f, ax = plt.subplots(figsize=(10, 10))
# plt.title('Question features Correlation Matrix',fontsize=20)

# sns.heatmap(train_df[num_feat].corr(),square=True, linewidths=0.25,vmax=0.7,cmap="YlGnBu",
#             linecolor='w',annot=True,annot_kws={"size":10},mask=mask,cbar_kws={"shrink": .9});

In [ ]:
plt.figure(figsize=(15, 5))

target_correlation = train_df.corr()['target'][1:]
plt.plot(target_correlation)

# **Data Preprocessing**

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [ ]:
train_df['question']

In [ ]:
def qes_preprocessing(qes):
    # Data cleaning:
    qes = re.sub(re.compile('<.*?>'), '', qes)
    qes = re.sub('[^A-Za-z0-9]+', ' ', qes)

    # Lowercase:
    qes = qes.lower()

    # Tokenization:
    tokens = word_tokenize(qes)

    # Contractions replacement:
    tokens = [contraction_dict.get(token) if (contraction_dict.get(token) != None) else token for token in tokens]

    # Stop words removal:
    tokens = [w for w in tokens if w not in stop_words]

    # Stemming:
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Lemmatization:
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    # Join words after preprocessed:
    qes = ' '.join(tokens) 
    
#     aug = naw.ContextualWordEmbsAug(
#     model_path='bert-base-uncased', action="insert")
#     augmented_text = aug.augment(text)

    return qes

In [ ]:
train_df['preprocessed_questions'] = train_df['question'].apply(qes_preprocessing)
train_df['preprocessed_questions']

# **Train model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score, confusion_matrix, classification_report

X = train_df['preprocessed_questions']
y = train_df.target

**Vectorizing**

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, analyzer=lambda x: x, min_df=0.01, max_df=0.999)
# min_df & max_df param added for less memory usage

tf_idf = vectorizer.fit_transform(X).toarray()
pd.DataFrame(tf_idf, columns=vectorizer.get_feature_names()).head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf, y, test_size=0.2, random_state=42)

**Model 1: MultinomialNB**

In [ ]:
# # BOW
# nb_bow_pipeline = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(2,4), max_df=0.85)),
#                      ("model", MultinomialNB())])

# # TF-IDF
# nb_tdf_pipelione = Pipeline([("tfid", TfidfVectorizer(lowercase=False, min_df=0.01, max_df=0.95)),
#                      ("model", MultinomialNB())])

**Model 2: Logistic Regression**

In [ ]:
# # BOW
# lr_bow_pipeline = Pipeline([("cv", CountVectorizer(analyzer="word", ngram_range=(1,4), max_df=0.9)),
#                      ("model", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=250, verbose=1, n_jobs=-1))
#                            ])

# # TF-IDF
# lr_tdf_pipeline = Pipeline([("tfid", TfidfVectorizer(lowercase=False, min_df=0.01, max_df=0.95)),
#                      ("model", LogisticRegression(solver="saga", class_weight="balanced", C=0.45, max_iter=250, verbose=1, n_jobs=-1))
#                            ])

**Model 3: XGBoost Classifier without weigths**

In [ ]:
def get_fscore_matrix(fitted_clf, model_name):
    print(model_name, ' :')
    
    # get classes predictions for the classification report 
    y_train_pred, y_pred = fitted_clf.predict(X_train), fitted_clf.predict(X_test)
    print(classification_report(y_test, y_pred), '\n') # target_names=y
    
    # computes probabilities keep the ones for the positive outcome only      
    print(f'F1-score = {f1_score(y_test, y_pred):.2f}')

In [ ]:
from xgboost import XGBClassifier

XGBmodel = XGBClassifier(objective="binary:logistic")
XGBmodel.fit(X_train, y_train)
get_fscore_matrix(XGBmodel, 'XGB Clf withOUT weights')

**Model 4: XGBoost Classifier with weigths**

In [ ]:
ratio = ((len(y_train) - y_train.sum()) - y_train.sum()) / y_train.sum()
ratio

In [ ]:
WXGBmodel = XGBClassifier(objective="binary:logistic", scale_pos_weight=ratio)
WXGBmodel.fit(X_train, y_train)
get_fscore_matrix(WXGBmodel, 'XGB Clf WITH weights')

**Model 5: LGBM with weights**

In [ ]:
import lightgbm as lgb

LGBMmodel = lgb.LGBMClassifier(n_jobs = -1, class_weight={0:y_train.sum(), 1:len(y_train) - y_train.sum()})
LGBMmodel.fit(X_train, y_train)
get_fscore_matrix(LGBMmodel, 'LGBM weighted')

**Add model to model list**

In [ ]:
# models = [nb_bow_pipeline, nb_tdf_pipelione, lr_bow_pipeline, lr_tdf_pipeline]

**Plot Vectorize matrix**

In [ ]:
# import matplotlib.pyplot as plt  
# from sklearn.datasets import make_classification
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC

# # X, y = make_classification(random_state=0)
# # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# # clf = SVC(random_state=0)
# # clf.fit(X_train, y_train)
# # SVC(random_state=0)
# # plot_confusion_matrix(clf, X_test, y_test, normalize=None, cmap=plt.cm.Blues)
# # plt.show()

# vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
# vectorized = vectorizer.fit_transform(X)
# pd.DataFrame(vectorized.toarray(), 
#             index=['sentence '+str(i) 
#                    for i in range(1, 1+len(X))],
#             columns=vectorizer.get_feature_names())

# # vectorizer = TfidfVectorizer()
# # X = vectorizer.fit_transform(X)
# # print(vectorizer.get_feature_names())
# # print(X.shape)

In [ ]:
# kfold = KFold(n_splits=2, shuffle=False)

# fold = 1

# for train_index, test_index in kfold.split(X, y):
#     X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index] 
#     print(X_train)
#     print(f'Fold {fold}:')
#     for model_pipeline in models:
#         print(model_pipeline)
#         model_pipeline.fit(X_train, y_train)
# #         predictions = model_pipeline.predict_log_proba(X_test)

#         print(classification_report(y_test, predictions), '\n') 
        
#         print(f'F1-score = {f1_score(y_test, predictions):.2f}')

#     fold += 1

# **2. Load test.csv, make predictions and output submission**

**2.1 Load dataset**

In [ ]:
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.shape

**2.2 Preprocessing data**

In [ ]:
test_df['preprocessed'] = test_df['question_text'].apply(qes_preprocessing)

**2.3 Make predictions**

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False, analyzer=lambda x: x, min_df=0.01, max_df=0.999)
# min_df & max_df param added for less memory usage

tf_idf = vectorizer.fit_transform(test_df['preprocessed']).toarray()
pd.DataFrame(tf_idf, columns=vectorizer.get_feature_names()).head()

In [ ]:
# X_test, y_test = train_test_split(tf_idf, y, test_size=0.2, random_state=42)

In [ ]:
# predictions = models[2].predict(test_df['preprocessed'])

In [ ]:
prediction1 = XGBmodel.predict(tf_idf)

In [ ]:
prediction2 = WXGBmodel.predict(tf_idf)

In [ ]:
prediction3 = LGBMmodel.predict(tf_idf)

In [ ]:
print(len(prediction1))
print(prediction1[-10:])
print(test_df['question_text'][-10:])

**2.4 Make submission**

In [ ]:
test_df['prediction'] = prediction1
results = test_df[['qid', 'prediction']]
results.to_csv('submission.csv', index=False)
results.shape